# MLflow

MLflow is an open source tool that manages machine learning workflows. 

## Why MLflow? 
- Track experiments by logging hyperparameters, parameters, metrics, etc.
- Collaborate on and compare experiments.
- Share data and models. 
- Easier to deploy models - making the developement and production cycle tighter.

## Installation

```bash
pip install mlflow
```

## Setup

```bash
mlflow server
```

This will launch an mlflow tracking server at `http://127.0.0.1:5000`. Which will record your experiments, runs and models.

:::{.callout-note}
This setup launches mlflow locally, to setup mlflow for a shared space follow [the documentation](https://mlflow.org/docs/latest/tracking.html#tracking-server)
:::

if you open http://127.0.0.1:5000 in your browser, you would see the following.

![](mlflow_ss_1.png)


## New Experiment

In [1]:
# sample dataset and model

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1000, class_sep=0.3, random_state=2)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=0
)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X_train[:2], "\n", y_train[:2])

(670, 20) (330, 20) (670,) (330,)
[[-0.78669445  0.50422194  0.14499686  0.27023408 -0.41568319  2.29765363
  -0.78492925  0.04416244  0.01998036 -1.56917287  1.63286779 -0.89135744
   0.08933603 -0.80613711  0.05557609  1.36246637  2.43942101 -1.00591414
  -1.80253155 -1.22509769]
 [ 0.7902646   1.58800122 -1.22745766  2.45455804  1.41597967 -0.9577786
  -0.07197365  0.26854757 -0.13132486  1.30857502  0.28586077 -2.27043702
   0.01217551  0.97624736  1.23833836 -0.19059581  0.05277705  0.45574733
   0.23691436 -0.78250385]] 
 [0 1]


In [2]:
import mlflow

In [3]:
# connect to the mlflow server
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000/"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [4]:
# create experiment and get experiment ID
experiment_name = "testing_mlflow"
experiment_id = mlflow.create_experiment(experiment_name)

if you go back to the mlflow page, you can see a new experiment created with the name.

## Log Parameters and Metrics

Every experiment contains multiple runs. Each run start by `mlflow.start_run()`

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [6]:
with mlflow.start_run(experiment_id=experiment_id):
    
    penalty = "l2"
    C = 0.01
    
    clf = LogisticRegression(penalty=penalty, C=C)
    clf.fit(X_train, y_train)
    
    train_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
    test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])    
    
    mlflow.log_params({"penalty":penalty, "C":C}) # logs hyperparameters
    mlflow.log_metrics({"train_auc":train_auc, "test_auc":test_auc}) # logs metrics

In [7]:
# without `with` statement 
mlflow.start_run(experiment_id=experiment_id)

penalty = "l2"
C = 0.001

clf = LogisticRegression(penalty=penalty, C=C)
clf.fit(X_train, y_train)

train_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])    

mlflow.log_params({"penalty":penalty, "C":C}) # logs hyperparameters
mlflow.log_metrics({"train_auc":train_auc, "test_auc":test_auc}) # logs metrics

mlflow.end_run() # be sure to end the run

The experiment should have new runs ready to be viewed. 

### Hyperparameter Search

Any hyperparameter search can be logged to MLflow. (Including Optuna: [Guide](../optuna/optuna.ipynb), [MLflow-Optuna-Integration](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.MLflowCallback.html))

In [8]:
for c_value in [0.001, 0.01, 0.1, 1, 10]:
    
    with mlflow.start_run(experiment_id=experiment_id):
    
        penalty = "l2"
        C = c_value

        clf = LogisticRegression(penalty=penalty, C=C)
        clf.fit(X_train, y_train)

        train_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
        test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])    

        mlflow.log_params({"penalty":penalty, "C":C})
        mlflow.log_metrics({"train_auc":train_auc, "test_auc":test_auc}) 

### Explore Logged Experiment

In [9]:
runs = mlflow.search_runs(experiment_id) # returns a dataframe

print(f"{type(runs) = }") 
runs

type(runs) = <class 'pandas.core.frame.DataFrame'>


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.train_auc,metrics.test_auc,params.penalty,params.C,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name
0,c6cc6209735d49d8ba344db544de8c6f,1,FINISHED,./mlruns/1/c6cc6209735d49d8ba344db544de8c6f/ar...,2022-05-17 03:18:53.686000+00:00,2022-05-17 03:18:53.711000+00:00,0.684686,0.649352,l2,10,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...
1,fd14937f4ba14e4f99b397a7fd911b15,1,FINISHED,./mlruns/1/fd14937f4ba14e4f99b397a7fd911b15/ar...,2022-05-17 03:18:53.652000+00:00,2022-05-17 03:18:53.678000+00:00,0.684686,0.649352,l2,1,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...
2,e68fa76fe03a41d9a61c0932986208df,1,FINISHED,./mlruns/1/e68fa76fe03a41d9a61c0932986208df/ar...,2022-05-17 03:18:53.619000+00:00,2022-05-17 03:18:53.646000+00:00,0.684596,0.650862,l2,0.1,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...
3,5ad09dce7f784075880caa9db33b6c18,1,FINISHED,./mlruns/1/5ad09dce7f784075880caa9db33b6c18/ar...,2022-05-17 03:18:53.589000+00:00,2022-05-17 03:18:53.613000+00:00,0.684177,0.657014,l2,0.01,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...
4,a659f2aff5904a38bf55582f356a6a34,1,FINISHED,./mlruns/1/a659f2aff5904a38bf55582f356a6a34/ar...,2022-05-17 03:18:53.553000+00:00,2022-05-17 03:18:53.582000+00:00,0.681760,0.668435,l2,0.001,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...
5,ac488b27fd284e63b0fd09777a322726,1,FINISHED,./mlruns/1/ac488b27fd284e63b0fd09777a322726/ar...,2022-05-17 03:18:52.679000+00:00,2022-05-17 03:18:52.708000+00:00,0.681760,0.668435,l2,0.001,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...
6,767e1f0fdaa644d5bd478c5899fb3ddc,1,FINISHED,./mlruns/1/767e1f0fdaa644d5bd478c5899fb3ddc/ar...,2022-05-17 03:18:52.442000+00:00,2022-05-17 03:18:52.475000+00:00,0.684177,0.657014,l2,0.01,LOCAL,Work,/Users/Work/anaconda3/envs/mlogs/lib/python3.8...


## Log Model

In [10]:
with mlflow.start_run(experiment_id=experiment_id) :

    penalty = "l2"
    C = 0.001

    clf = LogisticRegression(penalty=penalty, C=C)
    clf.fit(X_train, y_train)

    train_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
    test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])    

    mlflow.log_params({"penalty":penalty, "C":C}) 
    mlflow.log_metrics({"train_auc":train_auc, "test_auc":test_auc}) 
    
    mlflow.sklearn.log_model(clf, "log_res") # logs model with model name

/Users/Work/anaconda3/envs/mlogs/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


This save the model file (in pickle format) and also saves `conda.yaml` file that contains the environment requirements and `MLmodel` which is a yaml file that contains instructions for MLflow to run this model.


![](mlflow_ss_3.png)

### Loading Logged Model

In [11]:
logged_model = "runs:/29b59f6a1ba84859b299569cc3f1b004/log_res"
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [12]:
loaded_model.predict(X_test)[:5]

array([1, 0, 1, 0, 0])

MLflow's API only contains `predict` function but it is possible for that work as `predict_proba`.

```python 
class SklearnModelWrapper(mlflow.pyfunc.PythonModel):
  def __init__(self, model):
    self.model = model
    
  def predict(self, context, model_input):
    return self.model.predict_proba(model_input)[:,1]
```
[refer here](https://docs.databricks.com/_static/notebooks/mlflow/mlflow-end-to-end-example.html)

:::{.callout-note}
If the mlflow is in a shared space, it is possible for anyone having access to the mlflow server to load the model easily. This is one of the hallmarks of mlflow.
:::

## Logging Artifacts

Artifacts are any file or directory that you would like to record as part of a run. Artifacts can be logged as any formats.  
 
For example : 
- graphs ( as images )  
- tables/dataframe ( as html tables df.to_html() )
- datasets  (csvs, parquets)
- feature list  (txt) 
- git-commit ids  

In [13]:
from pathlib import Path

import pandas as pd

data_dir = str(Path.cwd()) + "/data"
Path(data_dir).mkdir(exist_ok=True)

pd.DataFrame(X_train).to_csv(f"{data_dir}/X_train.csv", index=False)
pd.DataFrame(X_test).to_csv(f"{data_dir}/X_test.csv", index=False)

pd.DataFrame(y_train).to_csv(f"{data_dir}/y_train.csv", index=False)
pd.DataFrame(y_test).to_csv(f"{data_dir}/y_test.csv", index=False)

In [14]:
with mlflow.start_run(experiment_id=experiment_id) :

    penalty = "l2"
    C = 0.001

    clf = LogisticRegression(penalty=penalty, C=C)
    clf.fit(X_train, y_train)

    train_auc = roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])
    test_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])    

    mlflow.log_params({"penalty":penalty, "C":C}) 
    mlflow.log_metrics({"train_auc":train_auc, "test_auc":test_auc}) 
    
    mlflow.sklearn.log_model(clf, "log_res")
    
    mlflow.log_artifacts(data_dir, artifact_path="data") # logs the entire directory

:::{.callout-note}
This will copy all the contents of the directory to mlflow server. If the server is in a shared space, anyone having access can now download the artifacts for experimentation or validation.
:::

### Downloading Artifacts¶

In [15]:
# downloads the `data` artifact from the previously recorded run
from mlflow.tracking import MlflowClient

client = MlflowClient()

download_dir = str(Path.cwd()) + "/data/download"
Path(download_dir).mkdir(exist_ok=True)

artifact_path = client.download_artifacts(
    "1694c57a0b674342b88270b49702e551", "data", download_dir
)

In [16]:
X_train_downloaded = pd.read_csv(f"{artifact_path}/X_train.csv")
print(X_train_downloaded.shape)

(670, 20)


## Registering Model

Registering model requires the MLflow server to be hosted with some kind of RDS for model registry data storage. Registering model is similar to logging them with the difference of giving it a name and version. Registered models go into the model registry and then can be easily [deployed](https://www.mlflow.org/docs/latest/models.html).